# Pooling

:label:`sec_pooling`



多くの場合、画像を処理する際に、隠れ表現の空間解像度を徐々に下げていき、ネットワークの上位に行くほど、各隠れノードが感受する（入力の）受容野が大きくなるように情報を集約していきたくなる。


多くの場合、最終的なタスクは画像について何らかのグローバルな質問をする、例えば *猫がいるか？*
そのため、最終層のノードは通常、入力全体に対して感度があるべきである。

徐々に情報を集約し、より粗いマップを生成することで、最終的にグローバルな表現を学習するというこの目標を達成しつつ、処理の中間層で畳み込み層の利点をすべて維持することができる。
さらに、（ :numref:`sec_conv_layer` で説明したように）エッジのような低レベルの特徴を検出するとき、我々はしばしば翻訳に対してある程度不変な表現にしたいと考える。
例えば，黒と白の境界が明瞭な画像 `X` を用いて，画像全体を1ピクセル右にシフトした場合，つまり， `Z[i, j] = X[i, j+1]`,
とすると、新しい画像 `Z` の出力は大きく異なるかもしれない。
エッジは1ピクセル分移動し、それに伴ってすべての活性化も移動する。
現実には、物体が全く同じ場所に存在することはほとんどない。
実際、三脚と静止したオブジェクトがあっても、シャッターの動きによるカメラの振動で、すべてが1ピクセルほどずれることがある（ハイエンドのカメラには、この問題に対処する特別な機能が搭載されている）。

このセクションでは、プーリングレイヤーを紹介する。
これは、位置に対する畳み込み層の感度を緩和することと、表現を空間的にダウンサンプリングすることの2つの目的を果たすものである。


## Maximum Pooling and Average Pooling

畳み込み層と同様に、プーリング演算子は、そのストライドに従って入力のすべての領域上でスライドされる固定のウィンドゥから成り、固定のウィンドゥによって横断される各位置に対して単一の出力を計算する（時々 *pooling window* として知られている）。
しかし、畳み込み層の入力とカーネルの相互相関計算とは異なり、プーリング層はパラメータを含まない（*フィルター*がない）。
その代わり、プーリング演算子は決定論的であり、典型的にはプーリングウィンドウ内の要素の最大値または平均値のいずれかを計算する。
これらの演算は *最大プーリング* (略して *max pooling*) と呼ばれる。
および *平均プーリング* とそれぞれ呼ばれる。

どちらの場合も、相互相関演算子と同様に、プーリング窓は入力配列の左上から始まり、左から右、上から下へと入力配列上をスライドしていくと考えることができる。
ウィンドウ内の入力部分配列の最大値または平均値（*max* または *average* のどちらのプーリングが採用されているかに依存）を計算します。

www.DeepL.com/Translator（無料版）で翻訳しました。


![Maximum pooling with a pooling window shape of $2\times 2$. The shaded portions represent the first output element and the input element used for its computation: $\max(0, 1, 3, 4)=4$](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/pooling.svg)

:label:`fig_pooling`


The output array in :numref:`fig_pooling` above has a height of 2 and a width of 2.
The four elements are derived from the maximum value of $\text{max}$:

$$
\max(0, 1, 3, 4)=4,\\
\max(1, 2, 4, 5)=5,\\
\max(3, 4, 6, 7)=7,\\
\max(4, 5, 7, 8)=8.\\
$$

A pooling layer with a pooling window shape of $p \times q$
is called a $p \times q$ pooling layer.
The pooling operation is called $p \times q$ pooling.

Let us return to the object edge detection example
mentioned at the beginning of this section.
Now we will use the output of the convolutional layer
as the input for $2\times 2$ maximum pooling.
Set the convolutional layer input as `X` and the pooling layer output as `Y`. Whether or not the values of `X[i, j]` and `X[i, j+1]` are different,
or `X[i, j+1]` and `X[i, j+2]` are different,
the pooling layer outputs all include `Y[i, j]=1`.
That is to say, using the $2\times 2$ maximum pooling layer,
we can still detect if the pattern recognized by the convolutional layer
moves no more than one element in height and width.

In the code below, we implement the forward computation
of the pooling layer in the `pool2d` function.
This function is similar to the `corr2d` function
in :numref:`sec_conv_layer`.
However, here we have no kernel, computing the output
as either the max or the average of each region in the input..

In [1]:
%use @file[../djl.json]
// %load ../utils/djl-imports

In [2]:
val manager = NDManager.newBaseManager();

fun pool2d(X: NDArray, poolShape: Shape, mode: String) : NDArray{
    
    val poolHeight = poolShape.get(0)
    val poolWidth = poolShape.get(1)
    
    val Y = manager.zeros(Shape(X.getShape().get(0) - poolHeight + 1, 
                                        X.getShape().get(1) - poolWidth + 1))
    for(i in 0 until Y.getShape().get(0)){
        for(j in 0 until Y.getShape().get(1)){
            
            if("max".equals(mode)){
                Y.set(NDIndex(i.toString()+","+j), 
                            X.get(NDIndex(i.toString() + ":" + (i + poolHeight) + ", " + j + ":" + (j + poolWidth))).max());
            }
            else if("avg".equals(mode)){
                Y.set(NDIndex(i.toString()+","+j),
                            X.get(NDIndex(i.toString() + ":" + (i + poolHeight) + ", " + j + ":" + (j + poolWidth))).mean());
            }
            
        }
    }
    
    return Y
}

We can construct the input array `X` in the above diagram to validate the output of the two-dimensional maximum pooling layer.

In [3]:
var X = manager.arange(9f).reshape(3,3)
pool2d(X, Shape(2,2), "max")

ND: (2, 2) cpu() float32
[[4., 5.],
 [7., 8.],
]


At the same time, we experiment with the average pooling layer.

In [4]:
pool2d(X, Shape(2,2), "avg");

ND: (2, 2) cpu() float32
[[2., 3.],
 [5., 6.],
]


## Padding and Stride

As with convolutional layers, pooling layers
can also change the output shape.
And as before, we can alter the operation to achieve a desired output shape
by padding the input and adjusting the stride.
We can demonstrate the use of padding and strides
in pooling layers via the two-dimensional maximum pooling layer `maxPool2dBlock`
shipped in DJL's `Pool` module.
We first construct an input data of shape `(1, 1, 4, 4)`,
where the first two dimensions are batch and channel.

In [5]:
var X = manager.arange(16f).reshape(1, 1, 4, 4);
X

ND: (1, 1, 4, 4) cpu() float32
[[[[ 0.,  1.,  2.,  3.],
   [ 4.,  5.,  6.,  7.],
   [ 8.,  9., 10., 11.],
   [12., 13., 14., 15.],
  ],
 ],
]


Below, we use a pooling window of shape `(3, 3)`,
and a stride shape of `(3, 3)`

In [6]:
// defining block specifying kernel and stride
val block = Pool.maxPool2dBlock(Shape(3, 3), Shape(3, 3));
block.initialize(manager, DataType.FLOAT32, Shape(1,1,4,4));

val parameterStore = ParameterStore(manager, false);
// Because there are no model parameters in the pooling layer, we do not need
// to call the parameter initialization function
block.forward(parameterStore, NDList(X), true).singletonOrThrow();

ND: (1, 1, 1, 1) cpu() float32
[[[[10.],
  ],
 ],
]


The stride and padding can be manually specified.

In [7]:
// redefining block shapes for kernel shape, stride shape and pad shape
val block = Pool.maxPool2dBlock(Shape(3,3), Shape(2,2), Shape(1,1));
// block forward method
block.forward(parameterStore, NDList(X), true).singletonOrThrow();

ND: (1, 1, 2, 2) cpu() float32
[[[[ 5.,  7.],
   [13., 15.],
  ],
 ],
]


Of course, we can specify an arbitrary rectangular pooling window
and specify the padding and stride for height and width, respectively.

In [8]:
// redefining block shapes for kernel shape, stride shape and pad shape
val block = Pool.maxPool2dBlock(Shape(2,3), Shape(2,3), Shape(1,2));
block.forward(parameterStore, NDList(X), true).singletonOrThrow();

ND: (1, 1, 3, 2) cpu() float32
[[[[ 0.,  3.],
   [ 8., 11.],
   [12., 15.],
  ],
 ],
]


## Multiple Channels

When processing multi-channel input data,
the pooling layer pools each input channel separately,
rather than adding the inputs of each channel by channel
as in a convolutional layer.
This means that the number of output channels for the pooling layer
is the same as the number of input channels.
Below, we will concatenate arrays `X` and `X+1`
on the channel dimension to construct an input with 2 channels.

In [9]:
X = X.concat(X.add(1f), 1);
X

ND: (1, 2, 4, 4) cpu() float32
[[[[ 0.,  1.,  2.,  3.],
   [ 4.,  5.,  6.,  7.],
   [ 8.,  9., 10., 11.],
   [12., 13., 14., 15.],
  ],
  [[ 1.,  2.,  3.,  4.],
   [ 5.,  6.,  7.,  8.],
   [ 9., 10., 11., 12.],
   [13., 14., 15., 16.],
  ],
 ],
]


As we can see, the number of output channels is still 2 after pooling.

In [10]:
val block = Pool.maxPool2dBlock(Shape(3,3), Shape(2,2), Shape(1,1));
block.forward(parameterStore, NDList(X), true).singletonOrThrow();

ND: (1, 2, 2, 2) cpu() float32
[[[[ 5.,  7.],
   [13., 15.],
  ],
  [[ 6.,  8.],
   [14., 16.],
  ],
 ],
]


## Summary

* Taking the input elements in the pooling window, the maximum pooling operation assigns the maximum value as the output and the average pooling operation assigns the average value as the output.
* One of the major functions of a pooling layer is to alleviate the excessive sensitivity of the convolutional layer to location.
* We can specify the padding and stride for the pooling layer.
* Maximum pooling, combined with a stride larger than 1 can be used to reduce the resolution.
* The pooling layer's number of output channels is the same as the number of input channels.


## Exercises

1. Can you implement average pooling as a special case of a convolution layer? If so, do it.
1. Can you implement max pooling as a special case of a convolution layer? If so, do it.
1. What is the computational cost of the pooling layer? Assume that the input to the pooling layer is of size $c\times h\times w$, the pooling window has a shape of $p_h\times p_w$ with a padding of $(p_h, p_w)$ and a stride of $(s_h, s_w)$.
1. Why do you expect maximum pooling and average pooling to work differently?
1. Do we need a separate minimum pooling layer? Can you replace it with another operation?
1. Is there another operation between average and maximum pooling that you could consider (hint: recall the softmax)? Why might it not be so popular?
